In [1]:
import os
import pandas as pd
import socket
import sys

from sqlalchemy import text

def get_path():
    # получаем имя компьютера
    hostname = socket.gethostname()

    if hostname == '304-007':
        path_ = r'C:\Users\Public\dictionaries'
    else:
        path_ = os.path.expanduser('~\\Documents\\')

    return path_

path_ = get_path()
sys.path.append(path_)

from dict_sql_queries import dict_sql_queries
from paths_and_constants import cinfo, tup_server_1_big
from user_functions import create_sql_connection, get_sql_user_info_from_txt_file



Начинаю загружать пути

hostname = "304-004"

Файл с авторизационными данными пользователя (роль) "sql_user_info.txt" найден по адресу "C:\Users\VAIO\Documents\sql_user_info.txt"


In [ ]:
# список ролей для обычных привилегий
tup_roles = (
    'role_0',
    'role_1',
    'role_2',
    'role_3',
    'role_4',
    'role_5',
    'selector',
)

# список ролей, у которых не отбираем привилегии на отдельные таблицы
tup_roles_admins = (
    'role_0',
    'role_1',
)

next_row = '\n'

In [3]:
# отдельные таблицы для ограниченного доступа
dict_table_names_vs_table_schemas_revoke = {
    # dictionaries
    'mhcd_timing':      'dictionaries',
    '"system"':         'uploadingsuploadings_shedule',
    '"system"':         'uploadingsuploadings_status',
    # recipes

    # big_data
}

In [4]:
def run_the_query(query_key, conn, i=-1, txt_to_print="", column_name=None, **dict_attrib):
    # Формирование SQL-запроса
    query = dict_sql_queries[query_key].format_map(
        dict_attrib,  # подстановка атрибутов: имён схем, таблиц, ролей...
    )
    # Выполнение запроса
    if not column_name is None:
        ls_ = pd.read_sql_query(query, conn)[column_name].tolist()
        # Применение изменений (если транзакции используются явно)
        conn.commit()
        return ls_
    conn.execute(text(query))
    # Применение изменений (если транзакции используются явно)
    conn.commit()
    # отступ перед первым сообщением блока
    print(f'{("", next_row)[i == 0]}', end='')
    # Вывод сообщения о том, что...
    print(txt_to_print.format_map(
        dict_attrib,  # подстановка атрибутов: имён схем, таблиц, ролей...
    )
    )  # <---------- можно закомментировать вывод



In [5]:
# Функция, которая выдает права на использование схемы и на чтение таблиц для различных ролей
def grant_usage_on_schema():

    # Извлечение учетных данных (имя пользователя и пароль) из текстового файла
    [user, pwd] = get_sql_user_info_from_txt_file(cinfo)
    # user = 'fed'
    # pwd = '1111'

    # Создание соединения с базой данных с использованием учетных данных и сервера
    engine = create_sql_connection(user, pwd, tup_server_1_big)

    # Открытие контекста соединения
    with engine.connect() as conn:

        # Получение списка имён не-системных схем
        ls_schemas = run_the_query(
            'get_schema_names_raw', conn,
            column_name='schema_name', **{},
        )
        
        print(f'ls_schemas = {ls_schemas}')

        # Проход по списку, который содержит имена схем
        for table_schema in ls_schemas:

            # Проход по ролям (переменная tup_roles содержит список ролей)
            for i, role in enumerate(tup_roles[:]):

                # Выдача прав на использование схемы
                run_the_query(
                    'grant_usage_on_schema_raw', conn, i,
                    'Права на использование схемы {table_schema} переданы {role}',
                    **{
                        'table_schema': table_schema,  # подстановка имени схемы
                        'role': role,                  # подстановка роли
                    }
                )

            # Получение списка имён таблиц схемы
            ls_tables = ls_schemas = run_the_query(
            'get_table_names_raw', conn,
            column_name='table_name', **{'schema_name': table_schema},
        )

            # Проход по таблицам (переменная ls_tables содержит список таблиц)
            for table_name in ls_tables:

                if (table_name, table_schema) not in dict_table_names_vs_table_schemas_revoke.items():

                    # Проход по ролям (переменная tup_roles содержит список ролей)
                    for i, role in enumerate(tup_roles[:]):

                        # Выдача прав на чтение таблицы
                        run_the_query(
                            'grant_reading_privileges_raw', conn, i,
                            'Права на чтение таблицы {table_name} переданы {role}',
                            **{
                                'table_schema': table_schema,  # подстановка имени схемы
                                'table_name': table_name,      # подстановка имени таблицы
                                'role': role,                  # подстановка роли
                            }
                        )


In [ ]:

grant_usage_on_schema()
